<a href="https://www.kaggle.com/code/dalepeh/mcu-plotly?scriptVersionId=116016471" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mcu-movies-dataset/rating.csv
/kaggle/input/mcu-movies-dataset/box_office.csv
/kaggle/input/mcu-movies-dataset/creators.csv


# Import Libraries

In [2]:
import pandas as pd
import numpy as np

# Load Data

In [3]:
BOX_OFFICE = pd.read_csv('/kaggle/input/mcu-movies-dataset/box_office.csv')
RATING = pd.read_csv('/kaggle/input/mcu-movies-dataset/rating.csv')

## Data Cleaning

**1. BOX_OFFICE**

In [4]:
BOX_OFFICE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Phase                               30 non-null     object
 1   Film                                30 non-null     object
 2   U.S. release date                   30 non-null     object
 3   Box office gross U.S. and Canada    30 non-null     int64 
 4   Box office gross Other territories  30 non-null     int64 
 5   Box office gross Worldwide          30 non-null     int64 
 6   All-time ranking U.S. and Canada    30 non-null     int64 
 7   All-time ranking Worldwide          30 non-null     int64 
 8   Budget                              30 non-null     object
dtypes: int64(5), object(4)
memory usage: 2.2+ KB


In [5]:
# rename columns
BOX_OFFICE.rename(columns={'U.S. release date': 'Release Date',
                           'Box office gross U.S. and Canada': 'North America Box Office',
                           'Box office gross Other territories': 'Other Countries Box Office', 
                           'Box office gross Worldwide': 'Worldwide Box Office',
                           'All-time ranking U.S. and Canada': 'Ranking US',
                           'All-time ranking Worldwide': 'Ranking Worldwide'}, 
                      inplace=True)

In [6]:
#remove '$' and replace 'million' with '000000'
BOX_OFFICE['Budget'] = BOX_OFFICE['Budget'].str.replace('$', '', regex=True)

# remove 'space'
BOX_OFFICE['Budget'] = BOX_OFFICE['Budget'].str.replace('\s+', '', regex=True)

#remove million
BOX_OFFICE['Budget'] = BOX_OFFICE['Budget'].str.replace('million', '000000', regex=True)

#change 'Budget' column to numeric
BOX_OFFICE['Budget'] = pd.to_numeric(BOX_OFFICE['Budget'])

In [7]:
#change 'date' column to datetime series

BOX_OFFICE['Release Date'] = pd.to_datetime(BOX_OFFICE['Release Date'])

In [8]:
BOX_OFFICE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Phase                       30 non-null     object        
 1   Film                        30 non-null     object        
 2   Release Date                30 non-null     datetime64[ns]
 3   North America Box Office    30 non-null     int64         
 4   Other Countries Box Office  30 non-null     int64         
 5   Worldwide Box Office        30 non-null     int64         
 6   Ranking US                  30 non-null     int64         
 7   Ranking Worldwide           30 non-null     int64         
 8   Budget                      30 non-null     int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 2.2+ KB


In [9]:
BOX_OFFICE.head(5)

,Phase,Film,Release Date,North America Box Office,Other Countries Box Office,Worldwide Box Office,Ranking US,Ranking Worldwide,Budget
0,Phase One,Iron Man,2008-05-02,319034126,266762121,585796247,79,176,140000000
1,Phase One,The Incredible Hulk,2008-06-13,134806913,129964083,264770996,465,593,150000000
2,Phase One,Iron Man 2,2010-05-07,312433331,311500000,623933331,85,158,200000000
3,Phase One,Thor,2011-05-06,181030624,268295994,449326618,262,268,150000000
4,Phase One,Captain America: The First Avenger,2011-07-22,176654505,193915269,370569774,279,367,140000000


**1.1 Highest Grossing Film Worldwide**

In [10]:
#Index the Highest Grossing Box Office

Highest_Grossing = BOX_OFFICE['Worldwide Box Office'].idxmax()

In [11]:
#Results of highest grossing film name and total worldwide take-in amount
hi_gross_world= BOX_OFFICE['Film'][Highest_Grossing]
hi_gross_box = BOX_OFFICE['Worldwide Box Office'][Highest_Grossing]

In [12]:
print('The Highest Grossing film is {} racking worldwide box office with ${:,}'.format(hi_gross_world, hi_gross_box))

The Highest Grossing film is Avengers: Endgame racking worldwide box office with $2,797,800,564


**1.2 Lowest Grossing Film Worldwide**

In [13]:
# Get Index for lowest grossing film
Lowest_Grossing = BOX_OFFICE['Worldwide Box Office'].idxmin()

In [14]:
#Results of lowest grossing film name and total worldwide take-in amount
lo_gross_world= BOX_OFFICE['Film'][Lowest_Grossing]
lo_gross_box = BOX_OFFICE['Worldwide Box Office'][Lowest_Grossing]

In [15]:
print('The Lowest Grossing film is {}, racking worldwide box office with only ${:,}'.format(lo_gross_world, lo_gross_box))

The Lowest Grossing film is The Incredible Hulk, racking worldwide box office with only $264,770,996


**1.3 Most Expensive MCU Film Ever Made**

In [16]:
# Get highest Budget Film

Highest_Budget = BOX_OFFICE['Budget'].idxmax()

In [17]:
#Results of highest budget film name and budget spent
hi_budget_film= BOX_OFFICE['Film'][Highest_Budget]
hi_budget = BOX_OFFICE['Budget'][Highest_Budget]

In [18]:
print('The Most Expensive film produced to date is {} with and estimated gross budget of ${:,}'.format(hi_budget_film, hi_budget))

The Most Expensive film produced to date is Avengers: Age of Ultron with and estimated gross budget of $444,000,000


**2. RATING**

In [19]:
RATING.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Phase                     30 non-null     object
 1   Film                      30 non-null     object
 2   Critical Rotten Tomatoes  30 non-null     object
 3   Critical Metacritic       30 non-null     object
 4   Public CinemaScore        30 non-null     object
 5   Public PostTrak           30 non-null     object
dtypes: object(6)
memory usage: 1.5+ KB


In [20]:
# rename columns

RATING.rename(columns={'Critical Rotten Tomatoes': 'Rotten Tomatoes Rating',
                       'Critical Metacritic': 'Metacritic Rating',
                       'Public CinemaScore': 'CinemaScore Grade',
                       'Public PostTrak': 'PostTrak Rating', 
                       }, 
                      inplace=True)

In [21]:
#remove '% (XX reviews)' in Rotten Tomatoes

RATING['Rotten Tomatoes Rating'] = RATING['Rotten Tomatoes Rating'].str[:-15]

# remove '% (XX reviews)' in MetaCritic
RATING['Metacritic Rating'] = RATING['Metacritic Rating'].str[:-13]


In [22]:
RATING.head(5)

,Phase,Film,Rotten Tomatoes Rating,Metacritic Rating,CinemaScore Grade,PostTrak Rating
0,Phase One,Iron Man,94,79,A,—
1,Phase One,The Incredible Hulk,67,61,A−,—
2,Phase One,Iron Man 2,71,57,A,—
3,Phase One,Thor,77,57,B+,—
4,Phase One,Captain America: The First Avenger,79,66,A−,—


In [23]:
#remove - and % in PostTrak
RATING['PostTrak Rating'] = RATING['PostTrak Rating'].str.replace('—', '', regex=True)

RATING['PostTrak Rating'] = RATING['PostTrak Rating'].str.replace('%', '', regex=True)

In [24]:
RATING['Rotten Tomatoes Rating'] = pd.to_numeric(RATING['Rotten Tomatoes Rating'])

RATING['Metacritic Rating'] = pd.to_numeric(RATING['Metacritic Rating'])

RATING['PostTrak Rating'] = pd.to_numeric(RATING['PostTrak Rating'])

In [25]:
RATING.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Phase                   30 non-null     object 
 1   Film                    30 non-null     object 
 2   Rotten Tomatoes Rating  30 non-null     int64  
 3   Metacritic Rating       30 non-null     int64  
 4   CinemaScore Grade       30 non-null     object 
 5   PostTrak Rating         16 non-null     float64
dtypes: float64(1), int64(2), object(3)
memory usage: 1.5+ KB


In [26]:
RATING.tail(5)

,Phase,Film,Rotten Tomatoes Rating,Metacritic Rating,CinemaScore Grade,PostTrak Rating
25,Phase Four,Eternals,47,52,B,78.0
26,Phase Four,Spider-Man: No Way Home,93,71,A+,96.0
27,Phase Four,Doctor Strange in the Multiverse of Madness,74,60,B+,82.0
28,Phase Four,Thor: Love and Thunder,64,57,B+,77.0
29,Phase Four,Black Panther: Wakanda Forever,84,67,A,93.0


**2.1 Highest Rating Film Worldwide**

In [27]:
# Highest Rating Film
Highest_Rating = RATING['Rotten Tomatoes Rating'].idxmax()

In [28]:
#Results of highest rating film name and rotten tomatoes rating
hi_rate_name= RATING['Film'][Highest_Rating]
hi_rate_rot = RATING['Rotten Tomatoes Rating'][Highest_Rating]

In [29]:
print('The Highest Rating film in Rotten Tomatoes is {} with rating of {}%'.format(hi_rate_name, hi_rate_rot))

The Highest Rating film in Rotten Tomatoes is Black Panther with rating of 96%


**2.2 Lowest Rating Film Worldwide**

In [30]:
#Lowest Rating Film

Lowest_Rating = RATING['Rotten Tomatoes Rating'].idxmin()

In [31]:
#Results of Lowest rating film name and rotten tomatoes rating
lo_rate_name= RATING['Film'][Lowest_Rating]
lo_rate_rot = RATING['Rotten Tomatoes Rating'][Lowest_Rating]

In [32]:
print('The Lowest Rating film in Rotten Tomatoes is {} with rating of {}%'.format(lo_rate_name, lo_rate_rot))

The Lowest Rating film in Rotten Tomatoes is Eternals with rating of 47%


# Plotly Graphs

In [33]:
import plotly.express as px
import plotly.graph_objects as go

In [34]:
import plotly.offline as pyo
import plotly.graph_objs as go

In [35]:
box_ofc_p1 = BOX_OFFICE.query("Phase=='Phase One'")
box_ofc_p2 = BOX_OFFICE.query("Phase=='Phase Two'")
box_ofc_p3 = BOX_OFFICE.query("Phase=='Phase Three'")
box_ofc_p4 = BOX_OFFICE.query("Phase=='Phase Four'")

In [36]:
trace1 = go.Bar(
    x=box_ofc_p1['Film'],      
    y=box_ofc_p1['Worldwide Box Office'],
    name = 'Phase 1',
    marker=dict(color='#FFD700')
)
trace2 = go.Bar(
    x=box_ofc_p2['Film'],      
    y=box_ofc_p2['Worldwide Box Office'],
    name = 'Phase 2',
    marker=dict(color='#9EA0A1')
)
trace3 = go.Bar(
    x=box_ofc_p3['Film'],      
    y=box_ofc_p3['Worldwide Box Office'],
    name = 'Phase 3',
    marker=dict(color='#CD7F32')
)
trace4 = go.Bar(
    x=box_ofc_p4['Film'],      
    y=box_ofc_p4['Worldwide Box Office'],
    name = 'Phase 4',
    marker=dict(color='#21618C')
)
data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    title='Worldwide Box Office by Phases'
)
fig = go.Figure(data=data, layout=layout)
fig.show()

**Overall box office, Phase 4 scored the lowest. Perhaps it is because of the pandemic period where lesser movie-goers go to the cinemas to watch the films. However, based on MCU fans, many find that the quality of Phase 4 is lower than the earlier phases.**

In [37]:
rating_p1 = RATING.query("Phase=='Phase One'")
rating_p2 = RATING.query("Phase=='Phase Two'")
rating_p3 = RATING.query("Phase=='Phase Three'")
rating_p4 = RATING.query("Phase=='Phase Four'")

In [38]:
rating1 = go.Bar(
    x=rating_p1['Film'],      
    y=rating_p1['Rotten Tomatoes Rating'],
    name = 'Phase 1',
    marker=dict(color='#FFD700')
)
rating2 = go.Bar(
    x=rating_p2['Film'],      
    y=rating_p2['Rotten Tomatoes Rating'],
    name = 'Phase 2',
    marker=dict(color='#9EA0A1')
)
rating3 = go.Bar(
    x=rating_p3['Film'],      
    y=rating_p3['Rotten Tomatoes Rating'],
    name = 'Phase 3',
    marker=dict(color='#CD7F32')
)
rating4 = go.Bar(
    x=rating_p4['Film'],      
    y=rating_p4['Rotten Tomatoes Rating'],
    name = 'Phase 4',
    marker=dict(color='#21618C')
)
data = [rating1, rating2, rating3, rating4]
layout = go.Layout(
    title='Rotten Tomatoes Rating by Phases'
)
fig = go.Figure(data=data, layout=layout)
fig.show()

**Phase 4 by far gotten the worst rating with most of the films scoring below 80&**